# Coursera Capstone Project
# The Battle of Neighborhoods - Final Report (Week 1 &  2)¶

In [1]:
# As a first step we import the necessary libraries:  
# !conda install -c conda-forge geopy --yes  # use this to install conda 
#conda update -n base -c defaults conda  # use this to update  conda
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import json # library to handle JSON files

import requests # library to handle requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


# import k-means from clustering stage
from sklearn.cluster import KMeans

# import wget 
import wget 

# import Beautifulsoup 
from bs4 import BeautifulSoup 

 
print('Libraries imported.') 

Libraries imported.


In [2]:
# Lets now find the coordinates of Clients existing business in Athens, Patriarchou Ioakim street. 

address = 'patriarchou ioakim , Athens'   
geolocator = Nominatim() 
location = geolocator.geocode(address)
kolonaki_latitude = location.latitude 
kolonaki_longtitude = location.longitude
print('The geograpical coordinate of {}  Athens-Kolonaki  home are {}, {}.'.format(address, kolonaki_latitude, kolonaki_longtitude))
print('The exact location of our address is :  {}.'.format(location))  


C:\Users\sboun\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of patriarchou ioakim , Athens  Athens-Kolonaki  home are 37.9779396, 23.7436643.
The exact location of our address is :  Πατριάρχου Ιωακείμ, Κολωνάκι, Συνοικία Κολωνακίου, Αθήνα, Δήμος Αθηναίων, Περιφερειακή Ενότητα Κεντρικού Τομέα Αθηνών, Περιφέρεια Αττικής, Αττική, 10675, Ελλάδα.


In [3]:
# Access the Foursquare website by using ID and secret number 

CLIENT_ID = 'GN1PKJNKKDJYVYW4V2CDOWTQWBW0TAE5GJJ3JCAA2QPXMO2V' #  Foursquare ID
CLIENT_SECRET = 'XGU2XZYWMZS4T2ZXQERKZWN3TT0BUQ1PMLUF0C2R3LAQLVFY' #  Foursquare Secret
VERSION = '20190330' # Foursquare API version


In [4]:
# Indicate in the Foursquare website the coordinates of the location, which you need to obtain information about venues

LIMIT = 200  # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    kolonaki_latitude, 
    kolonaki_longtitude, 
    radius, 
    LIMIT)
url # display URL details 

'https://api.foursquare.com/v2/venues/explore?&client_id=GN1PKJNKKDJYVYW4V2CDOWTQWBW0TAE5GJJ3JCAA2QPXMO2V&client_secret=XGU2XZYWMZS4T2ZXQERKZWN3TT0BUQ1PMLUF0C2R3LAQLVFY&v=20190330&ll=37.9779396,23.7436643&radius=1000&limit=200'

In [5]:
# Get the results from the Fousquare site in json format 
results = requests.get(url).json() 
#results 


In [6]:
# define function for restructuring/redefining what will be displayed in  column categories 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
venues = results['response']['groups'][0]['items'] # select relevant info that you want from Json file
infoVenues = json_normalize(venues) # convert JSON to Pandas DataFrame 

# select relavant columns from above data 
infoVenuesColumns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearbyVenues =infoVenues.loc[:, infoVenuesColumns]

# filter the category for each row
nearbyVenues['venue.categories'] = nearbyVenues.apply(get_category_type, axis=1)


# clean columns
nearbyVenues.columns = [col.split(".")[-1] for col in nearbyVenues.columns]


In [8]:
nearbyVenues.head(10)   # show first 10 rows 


,name,categories,lat,lng
0,MI-RO,Boutique,37.977960,23.742114
1,Jazz In Jazz,Whisky Bar,37.979117,23.742624
2,Άλ3ξ,Italian Restaurant,37.978392,23.741426
3,Rock 'n Roll,Bar,37.976286,23.744124
4,Τυροκομείο Κωσταρέλου,Cheese Shop,37.978002,23.743952
5,Δεξαμενή,Café,37.978979,23.741899
6,Πλατεία Δεξαμενής,Plaza,37.979044,23.742046
7,Museum of Cycladic Art (Μουσείο Κυκλαδικής Τέχ...,History Museum,37.975843,23.742231
8,Coco-Mat Hotel Athens,Hotel,37.978036,23.744162
9,Cake Kolonaki,Cupcake Shop,37.976747,23.742582


In [9]:
# show the most common category in Athens Kolonaki 
nearbyVenues.groupby('categories').count()  

,name,lat,lng
categories,,,
American Restaurant,1,1,1
Art Museum,1,1,1
Bar,4,4,4
Bistro,1,1,1
Bookstore,2,2,2
Boutique,4,4,4
Café,12,12,12
Cheese Shop,1,1,1
Chocolate Shop,1,1,1


In [10]:
# create map of Athens-Kolonaki using latitude and longitude values
map_kolonaki = folium.Map(location=[kolonaki_latitude, kolonaki_longtitude], zoom_start=16)

# add markers to map
for lat, long, name, cat in zip(nearbyVenues['lat'], nearbyVenues['lng'], nearbyVenues['name'], nearbyVenues['categories']):
    label = "Name: {} -- Category:{}".format(name, cat)   
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_kolonaki) 
    
map_kolonaki 

In [11]:
# target url for getting the postcodes of Canada 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url 

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [12]:
# Get access to wikipedia through Beautifulsoup 
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml") 

In [13]:
# print soup in order to find table
#print(soup.prettify()) 

In [14]:
# find and print table using soup 
table = soup.find('table',class_='wikitable sortable')
# table 

In [15]:
# find headers 
headers = table.find_all('th')
#headers 

In [16]:
# find table rows 
rows = table.find_all('tr')
#rows 

In [17]:
# create an array with Postcode, Borough and Neighbourhhood 
info = []

for i in rows:
    info.append([ii.text.strip() for ii in i.find_all('td')])
info[1:10] 

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', 'Downtown Toronto', "Queen's Park"],
 ['M8A', 'Not assigned', 'Not assigned']]

In [18]:
# Convert above data to pandas dataframe 
df = pd.DataFrame(info, columns=['Postal Code', 'Borough', 'Neighbourhood'])
df 

,Postal Code,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Downtown Toronto,Queen's Park
9,M8A,Not assigned,Not assigned


In [19]:
# exclude none from Postal 
df = df[~df['Postal Code'].isnull()]  
df 

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Downtown Toronto,Queen's Park
9,M8A,Not assigned,Not assigned
10,M9A,Queen's Park,Not assigned


In [20]:
# exclude Not assigned from Borough 
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True) 

C:\Users\sboun\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
# reset_indexset index 
df1 = df.reset_index() 

In [22]:
# Group by Postal code and combine data of 'Borough and 'Neighbourhood' separated by comma   
df2= df1.groupby('Postal Code').agg(lambda x: ','.join(x))
df2.shape

(103, 2)

In [23]:
# check if in Neighbourhood there is a "not assigned" description 
df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df2.shape 

(103, 2)

In [24]:
# reset index 
df3 = df2.reset_index()
df3 

,Postal Code,Borough,Neighbourhood
0,M1B,"Scarborough,Scarborough","Rouge,Malvern"
1,M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
2,M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,"Scarborough,Scarborough,Scarborough","East Birchmount Park,Ionview,Kennedy Park"
7,M1L,"Scarborough,Scarborough,Scarborough","Clairlea,Golden Mile,Oakridge"
8,M1M,"Scarborough,Scarborough,Scarborough","Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,"Scarborough,Scarborough","Birch Cliff,Cliffside West"


In [25]:
# exclude double counting or more  from Borough 
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [26]:
# show first five rows 
df3.head() 

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [27]:
# show dimensions of our list 
df3.shape 

(103, 3)

In [28]:
# Download coordinates and postal codes for Toronto from web using wget 

import wget 
url_get = wget.download( 'http://cocl.us/Geospatial_data/toronto_coordinates.csv') 
coordinates = pd.read_csv(url_get)
coordinates.head() 

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
# merge two data frames 
df4 = df3.set_index('Postal Code') 
coordinates1 = coordinates.set_index('Postal Code')
df5 = pd.concat([df4, coordinates1], axis=1, join='inner')
df5.index.name = 'PostalCode'
df5.reset_index(inplace=True) 
df5 

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [30]:
# show dimensions of table 
df5.shape 

(103, 5)

In [31]:
# find coordinaties of Toronto 
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="torontoN")
location = geolocator.geocode(address)
latitudeToronto = location.latitude
longitudeToronto = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitudeToronto, longitudeToronto)) 

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [32]:
# map toronto with Neighboorhoods 
mapToronto = folium.Map(location=[latitudeToronto, longitudeToronto], zoom_start=11)

for lat, long, post, borough, neigh in zip(df5['Latitude'], df5['Longitude'], df5['PostalCode'], df5['Borough'], df5['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#cc3139', 
        fill_opacity=0.8,
        ).add_to(mapToronto) 
    
mapToronto 

In [33]:
# Limit Borough's  to 'EastToronto', 'CentralToronto', 'DowntownToronto', 'WestToronto' name 
dff = ['EastToronto', 'CentralToronto', 'DowntownToronto', 'WestToronto'] 
df6 = df5[df5['Borough'].isin(dff)].reset_index(drop=True)
df6.head() 

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,EastToronto,The Beaches,43.676357,-79.293031
1,M4K,EastToronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,EastToronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,EastToronto,Studio District,43.659526,-79.340923
4,M4N,CentralToronto,Lawrence Park,43.728020,-79.388790


In [34]:
# show dimensions of new constraint table of Toronto 
df6.shape 

(39, 5)

In [35]:
# map the above Borough's 
mapTorontoCenter = folium.Map(location=[latitudeToronto, longitudeToronto], zoom_start=12)

for lat, long, post, borough, neigh in zip(df6['Latitude'], df6['Longitude'], df6['PostalCode'], df6['Borough'], df6['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#cc3139', 
        fill_opacity=0.8,
        ).add_to(mapTorontoCenter)
    
mapTorontoCenter  

In [36]:
# Using FourSquare API to explore the Borough's 
CLIENT_ID = 'GN1PKJNKKDJYVYW4V2CDOWTQWBW0TAE5GJJ3JCAA2QPXMO2V' # your Foursquare ID
CLIENT_SECRET = 'XGU2XZYWMZS4T2ZXQERKZWN3TT0BUQ1PMLUF0C2R3LAQLVFY' # your Foursquare Secret
VERSION = '20190330' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET) 

Your credentails:
CLIENT_ID: GN1PKJNKKDJYVYW4V2CDOWTQWBW0TAE5GJJ3JCAA2QPXMO2V
CLIENT_SECRET:XGU2XZYWMZS4T2ZXQERKZWN3TT0BUQ1PMLUF0C2R3LAQLVFY


In [37]:
# find venues in relevant Borough's/Neighborhoods of table df6  
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df6['Latitude'], df6['Longitude'], df6['PostalCode'],df6['Borough'], df6['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name'])) 


In [38]:
# Get a table Boroughs/Neighborhood in Toronto with relevant venues info 
venuesTable = pd.DataFrame(venues)
venuesTable.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venuesTable.shape)
venuesTable.head()  

(1699, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,EastToronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,EastToronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,EastToronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,EastToronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,EastToronto,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [39]:
# Analyze venues 

torontoCentralTable = pd.get_dummies(venuesTable[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
torontoCentralTable['PostalCode'] = venuesTable['PostalCode'] 
torontoCentralTable['Borough'] = venuesTable['Borough'] 
torontoCentralTable['Neighborhoods'] = venuesTable['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
allignColumns = list(torontoCentralTable.columns[-3:]) + list(torontoCentralTable.columns[:-3])
torontoCentralTable = torontoCentralTable[allignColumns]

print(torontoCentralTable.shape)
torontoCentralTable.head() 

(1699, 234)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,EastToronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,M4E,EastToronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [40]:
# Get frequency 
torontoCentralFrequency = torontoCentralTable.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()
print(torontoCentralFrequency.shape)
torontoCentralFrequency.head()  

(39, 234)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,EastToronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.00,0.00000,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [41]:
# Get Top venues  
topVenues = 5 

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(topVenues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
venuesNeighborhoodsFilter = pd.DataFrame(columns=columns)
venuesNeighborhoodsFilter['PostalCode'] = torontoCentralFrequency['PostalCode']
venuesNeighborhoodsFilter['Borough'] = torontoCentralFrequency['Borough']
venuesNeighborhoodsFilter['Neighborhoods'] = torontoCentralFrequency['Neighborhoods']

for ind in np.arange(torontoCentralFrequency.shape[0]):
    rowCategories = torontoCentralFrequency.iloc[ind, :].iloc[3:]
    rowCategoriesSorted = rowCategories.sort_values(ascending=False)
    venuesNeighborhoodsFilter.iloc[ind, 3:] = rowCategoriesSorted.index.values[0:topVenues]

venuesNeighborhoodsFilter.sort_values(freqColumns, inplace=True)
venuesNeighborhoodsFilter.head()   

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
27,M5V,DowntownToronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina
32,M6J,WestToronto,"Little Portugal,Trinity",Bar,Restaurant,Coffee Shop,Asian Restaurant,Vietnamese Restaurant
3,M4M,EastToronto,Studio District,Café,Coffee Shop,Gastropub,Brewery,Bakery
25,M5S,DowntownToronto,"Harbord,University of Toronto",Café,Sandwich Place,Restaurant,Bookstore,Japanese Restaurant
26,M5T,DowntownToronto,"Chinatown,Grange Park,Kensington Market",Café,Vietnamese Restaurant,Bar,Chinese Restaurant,Dumpling Restaurant


In [42]:
kclusters = 7  

torontoCentralFrequencyCluster = torontoCentralFrequency.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torontoCentralFrequencyCluster)

TorontoCentralClusterTable = df6    
TorontoCentralClusterTable['Cluster'] = kmeans.labels_
TorontoCentralClusterTable = TorontoCentralClusterTable.join(venuesNeighborhoodsFilter.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
TorontoCentralClusterTable.sort_values(['Cluster'] + freqColumns, inplace=True)
TorontoCentralClusterTable.head() 


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
27,M5V,DowntownToronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,0,Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina
32,M6J,WestToronto,"Little Portugal,Trinity",43.647927,-79.419750,0,Bar,Restaurant,Coffee Shop,Asian Restaurant,Vietnamese Restaurant
3,M4M,EastToronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Bakery
25,M5S,DowntownToronto,"Harbord,University of Toronto",43.662696,-79.400049,0,Café,Sandwich Place,Restaurant,Bookstore,Japanese Restaurant
26,M5T,DowntownToronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049,0,Café,Vietnamese Restaurant,Bar,Chinese Restaurant,Dumpling Restaurant


In [43]:
# create map Red
mapTorontoVenues = folium.Map(location=[latitudeToronto, longitudeToronto], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(TorontoCentralClusterTable['Latitude'], TorontoCentralClusterTable['Longitude'], TorontoCentralClusterTable['PostalCode'], TorontoCentralClusterTable['Borough'], TorontoCentralClusterTable['Neighbourhood'], TorontoCentralClusterTable['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon], 
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapTorontoVenues )
        
mapTorontoVenues 

    